# Women Who Code :: Build-Your-Own Dataset

Sometimes data scientists are handed a fully prepared and cleaned dataset, but this is rarely the case. Today's workshop will give you practice in building your own dataset from scratch. We will use public APIs and publically available data files to create a dataset of weather and population data that is ready for downstream uses.

In this workshop, we'll be collecting and organizing information for fictional visitors to our fictional 


In [1]:
import pandas as pd
import requests

# Part 1 :: Calling Public APIs

In this first section we will use several publically available APIs to collect information about fictional visitors to our website. The only information we directly collect about visitors is their IP address. Beyond that, we'll have to look to outside sources to pull in information to learn more about our visitors.

## Get your IP address

An IP address is a unique address that identifies a device on the internet or a local network. IP stands for "Internet Protocol," which is the set of rules governing the format of data sent via the internet or local network.

To find out your own IP address, you can make a call to the [ipify](https://www.ipify.org/) API, a simple public IP address API. This API does not require an account or API key.

In [3]:
# api endpoint
url = "https://api.ipify.org"

# request formatted response
params = {
    "format": "json"
}

resp = requests.get(url, params)

Congrats, you just made a call to the first API of this workshop! Let's take a closer look at the response, and see what information we've collected from it.

In [5]:
# http response status codes indicate whether the request has been successfully completed
resp.status_code

200

In [6]:
# can get response in a string format
resp.text

'{"ip":"3.94.219.192"}'

In [7]:
# or more usefully as a json dictionary
resp.json()

{'ip': '3.94.219.192'}

In [8]:
# let's hold on to ip address, and use it in some next steps
ip = resp.json()["ip"]

## Get location for an IP address

Find the location for a given IP address.

https://ip-api.com/

In [10]:
url = f"http://ip-api.com/json/{ip}"

resp = requests.get(url)

In [11]:
resp.status_code

200

In [12]:
resp.json()

{'status': 'success',
 'country': 'United States',
 'countryCode': 'US',
 'region': 'VA',
 'regionName': 'Virginia',
 'city': 'Ashburn',
 'zip': '20149',
 'lat': 39.0438,
 'lon': -77.4874,
 'timezone': 'America/New_York',
 'isp': 'Amazon Technologies Inc.',
 'org': 'AWS EC2 (us-east-1)',
 'as': 'AS14618 Amazon.com, Inc.',
 'query': '3.94.219.192'}

In [13]:
# pull out the lat/long fields, since we can look up info about this location

lat = resp.json()["lat"]
long = resp.json()["lon"]

In [14]:
def get_location_info(ip: str):
    """
    Given an IP address, return location information.
    """
    url = f"http://ip-api.com/json/{ip}"
    resp = requests.get(url)
    return resp.json()

# Get your local weather

Look up the current weather for a location.

https://open-meteo.com/en

In [15]:
url = "https://api.open-meteo.com/v1/forecast"

params = {
    "latitude": lat,
    "longitude": long,
    "current_weather": True,
    "format": "json"
}

resp = requests.get(url, params)

In [16]:
resp.status_code

200

In [17]:
resp.json()

{'elevation': 98.3125,
 'latitude': 39,
 'current_weather': {'windspeed': 11,
  'winddirection': 1,
  'time': '2022-03-17T16:00',
  'temperature': 12.4,
  'weathercode': 61},
 'longitude': -77.5,
 'utc_offset_seconds': 0,
 'generationtime_ms': 0.12803077697753906}

In [18]:
def get_weather_info(lat: float, long: float):
    """
    Given a latitude and longitude, return the current weather.
    """
    url = "https://api.open-meteo.com/v1/forecast"

    params = {
        "latitude": lat,
        "longitude": long,
        "current_weather": True,
        "format": "json"
    }

    resp = requests.get(url, params)
    return resp.json()["current_weather"]

# Organize into a function

Input: None (will read from your computer's IP)

Output: Dictionary of the data we want to store

In [32]:
def get_my_geo_weather_data():
    
    # get ip address for current computer
    ip_info = get_ip_info()
    
    # pull out ip for input to next call
    ip = ip_info["ip"]
    
    # get location info for the ip address
    location_info = get_location_info(ip)
    
    # pull out lat/long for input to next call
    lat = location_info["lat"]
    long = location_info["lon"]
    
    # get the current weather at the lat/long
    weather_info = get_weather_info(lat, long)
    
    # only works in python 3.9 and above
    all_data = ip_info | location_info | weather_info
    
    # alternative for lower python versions using unpacking
    # all_data = {**ip_info, **location_info, **weather_info}
    
    return all_data

In [33]:
get_my_geo_weather_data()

{'ip': '3.94.219.192',
 'status': 'success',
 'country': 'United States',
 'countryCode': 'US',
 'region': 'VA',
 'regionName': 'Virginia',
 'city': 'Ashburn',
 'zip': '20149',
 'lat': 39.0438,
 'lon': -77.4874,
 'timezone': 'America/New_York',
 'isp': 'Amazon Technologies Inc.',
 'org': 'AWS EC2 (us-east-1)',
 'as': 'AS14618 Amazon.com, Inc.',
 'query': '3.94.219.192',
 'winddirection': 10,
 'weathercode': 61,
 'windspeed': 11.6,
 'time': '2022-03-17T16:00',
 'temperature': 12.4}

# Use fake IP addresses

In [34]:
# pip install faker

In [35]:
from faker import Faker

In [36]:
faker = Faker()
faker.ipv4() 

'166.58.28.192'

In [37]:
def get_fake_geo_weather_data():
    
    # get ip address for current computer
    faker = Faker()
    ip_info = {"ip": faker.ipv4()}
    
    # pull out ip for input to next call
    ip = ip_info["ip"]
    
    # get location info for the ip address
    location_info = get_location_info(ip)
    
    # pull out lat/long for input to next call
    lat = location_info["lat"]
    long = location_info["lon"]
    
    # get the current weather at the lat/long
    weather_info = get_weather_info(lat, long)
    
    # only works in python 3.9 and above
    all_data = ip_info | location_info | weather_info
    
    # alternative for lower python versions using unpacking
    # all_data = {**ip_info, **location_info, **weather_info}
    
    return all_data

In [38]:
get_fake_geo_weather_data()

{'ip': '220.130.3.59',
 'status': 'success',
 'country': 'Taiwan',
 'countryCode': 'TW',
 'region': 'NWT',
 'regionName': 'New Taipei',
 'city': 'New Taipei',
 'zip': '',
 'lat': 24.9389,
 'lon': 121.5422,
 'timezone': 'Asia/Taipei',
 'isp': 'Chunghwa Telecom Co., Ltd.',
 'org': 'Chunghwa Telecom Co. Ltd.',
 'as': 'AS3462 Data Communication Business Group',
 'query': '220.130.3.59',
 'winddirection': 151,
 'temperature': 22.5,
 'windspeed': 1.1,
 'time': '2022-03-17T16:00',
 'weathercode': 2}

In [39]:
get_fake_geo_weather_data()

{'ip': '86.59.60.6',
 'status': 'success',
 'country': 'Austria',
 'countryCode': 'AT',
 'region': '9',
 'regionName': 'Vienna',
 'city': 'Vienna',
 'zip': '1210',
 'lat': 48.2641,
 'lon': 16.402,
 'timezone': 'Europe/Vienna',
 'isp': 'Hutchison Drei Austria GmbH',
 'org': 'SIL-NET / Austria',
 'as': 'AS8437 Hutchison Drei Austria GmbH',
 'query': '86.59.60.6',
 'winddirection': 123,
 'windspeed': 11,
 'weathercode': 1,
 'time': '2022-03-17T16:00',
 'temperature': 11.5}

In [40]:
get_fake_geo_weather_data()

{'ip': '172.15.61.109',
 'status': 'success',
 'country': 'United States',
 'countryCode': 'US',
 'region': 'CA',
 'regionName': 'California',
 'city': 'Modesto',
 'zip': '95356',
 'lat': 37.6976,
 'lon': -121.0369,
 'timezone': 'America/Los_Angeles',
 'isp': 'AT&T Services, Inc.',
 'org': 'Gay Nicholas',
 'as': 'AS7018 AT&T Services, Inc.',
 'query': '172.15.61.109',
 'weathercode': 3,
 'windspeed': 4.7,
 'winddirection': 94,
 'time': '2022-03-17T16:00',
 'temperature': 9.6}

In [41]:
get_fake_geo_weather_data()

{'ip': '134.208.37.207',
 'status': 'success',
 'country': 'Taiwan',
 'countryCode': 'TW',
 'region': '',
 'regionName': 'Taiwan',
 'city': 'Taichung',
 'zip': '',
 'lat': 24.1496,
 'lon': 120.672,
 'timezone': 'Asia/Taipei',
 'isp': 'Taiwan Academic Network (TANet) Information Center',
 'org': 'Acanet TWN',
 'as': 'AS1659 Taiwan Academic Network (TANet) Information Center',
 'query': '134.208.37.207',
 'weathercode': 95,
 'winddirection': 146,
 'time': '2022-03-17T16:00',
 'temperature': 21.7,
 'windspeed': 3.3}

# Generate Dataset

In [115]:
weather_dicts = [get_fake_geo_weather_data() for i in range(100)]

In [116]:
df_weather = pd.DataFrame(weather_dicts)

In [117]:
df_weather.head()

,ip,status,country,countryCode,region,regionName,city,zip,lat,lon,timezone,isp,org,as,query,temperature,weathercode,windspeed,winddirection,time
0,162.27.156.161,success,United Kingdom,GB,ENG,England,London,W1B,51.5074,-0.127758,Europe/London,Daisy Corporate Services Trading Ltd,R.R. Donnelley & Sons Company,AS5413 Daisy Corporate Services Trading Ltd,162.27.156.161,12.7,0,11.2,269,2022-03-17T17:00
1,161.204.84.119,success,United States,US,CA,California,San Francisco,94128,37.7749,-122.419000,America/Los_Angeles,Visa International Service Association,Visa International Service Association,,161.204.84.119,10.8,3,9.7,243,2022-03-17T17:00
2,15.204.187.139,success,United States,US,VA,Virginia,Reston,20190,38.9609,-77.342900,America/New_York,OVH SAS,OVH US LLC,AS16276 OVH SAS,15.204.187.139,12.9,80,10.8,359,2022-03-17T17:00
3,21.5.32.123,success,United States,US,OH,Ohio,Whitehall,43218,39.9747,-82.894700,America/New_York,DoD Network Information Center,DoD Network Information Center,AS749 DoD Network Information Center,21.5.32.123,18.8,0,5.4,252,2022-03-17T17:00
4,193.71.21.8,success,Norway,NO,30,Viken,Auli,1929,60.0421,11.365500,Europe/Oslo,GLOBALCONNECT,BROADNET,AS2116 GLOBALCONNECT AS,193.71.21.8,3.8,61,21.7,150,2022-03-17T17:00


In [118]:
df_weather["country"].unique()

array(['United Kingdom', 'United States', 'Norway', 'Russia', 'Canada',
       'Italy', 'Japan', 'Austria', 'Spain', 'China', 'Switzerland',
       'Mauritius', 'South Korea', 'Australia', 'Venezuela', 'Thailand',
       'France', 'Lithuania', 'Singapore', 'Hungary', 'Slovakia', 'Chile',
       'Germany', 'Mexico', 'Egypt', 'Vietnam', 'Taiwan', 'India',
       'Argentina', 'Netherlands', 'Israel'], dtype=object)

# Join with Migration Data

XLSX file available from the UN

https://population.un.org/wpp/Download/Standard/Migration/

In [119]:
df_migration = pd.read_excel("WPP2019_MIGR_F01_NET_MIGRATION_RATE.xlsx", skiprows=range(16))

In [120]:
df_migration.head()

,Index,Variant,"Region, subregion, country or area *",Notes,Country code,Type,Parent code,1950-1955,1955-1960,1960-1965,...,1970-1975,1975-1980,1980-1985,1985-1990,1990-1995,1995-2000,2000-2005,2005-2010,2010-2015,2015-2020
0,1,Estimates,WORLD,NaN,900,World,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Estimates,UN development groups,a,1803,Label/Separator,900,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,3,Estimates,More developed regions,b,901,Development Group,1803,0.031,0.032,0.533,...,1.269,1.235,1.062,1.235,1.838,2.228,2.745,2.773,2.332,2.215
3,4,Estimates,Less developed regions,c,902,Development Group,1803,-0.014,-0.014,-0.224,...,-0.456,-0.411,-0.327,-0.352,-0.486,-0.551,-0.64,-0.613,-0.491,-0.443
4,5,Estimates,Least developed countries,d,941,Development Group,902,-0.487,-0.567,-0.787,...,-2.699,-2.118,-2.927,-1.573,0.321,-1.389,-1.254,-2.42,-1.716,-0.973


In [121]:
df_migration.tail()

,Index,Variant,"Region, subregion, country or area *",Notes,Country code,Type,Parent code,1950-1955,1955-1960,1960-1965,...,1970-1975,1975-1980,1980-1985,1985-1990,1990-1995,1995-2000,2000-2005,2005-2010,2010-2015,2015-2020
250,251,Estimates,Netherlands,28,528,Country/Area,926,-1.637,-0.401,0.196,...,2.799,2.096,0.998,1.896,2.45,2.275,1.809,0.795,0.746,0.939
251,252,Estimates,Switzerland,NaN,756,Country/Area,926,4.947,5.078,9.993,...,0.807,-4.29,2.549,3.999,7.251,1.845,5.124,9.078,9.867,6.135
252,253,Estimates,NORTHERN AMERICA,NaN,918,SDG subregion,1829,1.58,2.743,1.899,...,3.083,3.376,2.887,3.094,3.588,6.354,3.992,4.027,3.541,3.296
253,254,Estimates,Canada,NaN,124,Country/Area,918,7.56,6.74,1.907,...,6.825,3.544,2.982,6.45,4.871,5.245,6.706,8.001,7.076,6.562
254,255,Estimates,United States of America,29,840,Country/Area,918,1.05,2.371,1.9,...,2.697,3.363,2.879,2.734,3.452,6.48,3.7,3.595,3.151,2.929


In [122]:
# limit to only country-level records
df_migration_countries = df_migration.loc[df_migration["Type"]=="Country/Area"]

df_migration_countries.head()

,Index,Variant,"Region, subregion, country or area *",Notes,Country code,Type,Parent code,1950-1955,1955-1960,1960-1965,...,1970-1975,1975-1980,1980-1985,1985-1990,1990-1995,1995-2000,2000-2005,2005-2010,2010-2015,2015-2020
26,27,Estimates,Burundi,NaN,108,Country/Area,910,-5.773,-5.252,-5.769,...,-14.847,-7.65,-6.248,-7.066,-11.203,-14.728,-0.719,0.748,-1.487,0.181
27,28,Estimates,Comoros,NaN,174,Country/Area,910,0,-6.666,-8.522,...,-4.514,7.078,-2.714,-2.347,-1.353,-2.358,-3.466,-3.074,-2.726,-2.429
28,29,Estimates,Djibouti,NaN,262,Country/Area,910,3.037,13.052,36.254,...,36.47,62.553,5.098,35.432,-14.745,2.964,-2.398,-3.011,1.369,0.947
29,30,Estimates,Eritrea,NaN,232,Country/Area,910,0.232,0.629,1.298,...,1.42,1.235,1.07,-3.754,-28.212,-11.564,17.76,-5.337,-15.108,-11.571
30,31,Estimates,Ethiopia,NaN,231,Country/Area,910,-0.21,-0.19,-0.17,...,-0.391,-12.479,1.312,3.461,5.557,-0.505,-0.421,-0.122,0.849,0.278


In [123]:
df_migration_subset = df_migration_countries[["Region, subregion, country or area *", "2015-2020"]]

In [124]:
df_migration_subset.head()

,"Region, subregion, country or area *",2015-2020
26,Burundi,0.181
27,Comoros,-2.429
28,Djibouti,0.947
29,Eritrea,-11.571
30,Ethiopia,0.278


In [125]:
df_migration_subset.rename({"Region, subregion, country or area *": "country", "2015-2020": "migration_rate"}, axis=1, inplace=True)

/Users/krr024/opt/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [126]:
df_migration_subset.head()

,country,migration_rate
26,Burundi,0.181
27,Comoros,-2.429
28,Djibouti,0.947
29,Eritrea,-11.571
30,Ethiopia,0.278


In [128]:
df_weather_migration = df_weather.merge(df_migration_subset, how="left", left_on="country", right_on="country")

In [129]:
df_weather_migration.head()

,ip,status,country,countryCode,region,regionName,city,zip,lat,lon,...,isp,org,as,query,temperature,weathercode,windspeed,winddirection,time,migration_rate
0,162.27.156.161,success,United Kingdom,GB,ENG,England,London,W1B,51.5074,-0.127758,...,Daisy Corporate Services Trading Ltd,R.R. Donnelley & Sons Company,AS5413 Daisy Corporate Services Trading Ltd,162.27.156.161,12.7,0,11.2,269,2022-03-17T17:00,3.898
1,161.204.84.119,success,United States,US,CA,California,San Francisco,94128,37.7749,-122.419000,...,Visa International Service Association,Visa International Service Association,,161.204.84.119,10.8,3,9.7,243,2022-03-17T17:00,NaN
2,15.204.187.139,success,United States,US,VA,Virginia,Reston,20190,38.9609,-77.342900,...,OVH SAS,OVH US LLC,AS16276 OVH SAS,15.204.187.139,12.9,80,10.8,359,2022-03-17T17:00,NaN
3,21.5.32.123,success,United States,US,OH,Ohio,Whitehall,43218,39.9747,-82.894700,...,DoD Network Information Center,DoD Network Information Center,AS749 DoD Network Information Center,21.5.32.123,18.8,0,5.4,252,2022-03-17T17:00,NaN
4,193.71.21.8,success,Norway,NO,30,Viken,Auli,1929,60.0421,11.365500,...,GLOBALCONNECT,BROADNET,AS2116 GLOBALCONNECT AS,193.71.21.8,3.8,61,21.7,150,2022-03-17T17:00,5.273


In [130]:
# some countries didn't match because names are different

df_weather_migration.loc[df_weather_migration["migration_rate"].isna(), "country"].unique()

array(['United States', 'Russia', 'South Korea', 'Venezuela', 'Vietnam',
       'Taiwan'], dtype=object)

In [131]:
# a few countries did match up

df_weather_migration.loc[df_weather_migration["migration_rate"].notna(), "country"].unique()

array(['United Kingdom', 'Norway', 'Canada', 'Italy', 'Japan', 'Austria',
       'Spain', 'China', 'Switzerland', 'Mauritius', 'Australia',
       'Thailand', 'France', 'Lithuania', 'Singapore', 'Hungary',
       'Slovakia', 'Chile', 'Germany', 'Mexico', 'Egypt', 'India',
       'Argentina', 'Netherlands', 'Israel'], dtype=object)

In [132]:
df_migration_subset.loc[df_migration_subset["country"].str.contains("United States")]

,country,migration_rate
164,United States Virgin Islands,-4.306
254,United States of America,2.929


In [133]:
df_migration_subset.loc[df_migration_subset["country"].str.contains("Syria")]

,country,migration_rate
103,Syrian Arab Republic,-24.08


In [134]:
df_migration_subset.loc[df_migration_subset["country"].str.contains("Korea")]

,country,migration_rate
130,Dem. People's Republic of Korea,-0.212
133,Republic of Korea,0.23


In [139]:
df_migration_subset.loc[df_migration_subset["country"].str.contains("Russia")]

,country,migration_rate
216,Russian Federation,1.254


In [140]:
df_migration_subset.loc[df_migration_subset["country"].str.contains("Venezuela")]

,country,migration_rate
187,Venezuela (Bolivarian Republic of),-22.327


In [142]:
df_migration_subset.loc[df_migration_subset["country"].str.contains("Viet")]

,country,migration_rate
145,Viet Nam,-0.842


In [149]:
df_migration_subset.loc[df_migration_subset["country"].str.contains("Taiwan")]

,country,migration_rate
129,"China, Taiwan Province of China",1.267


In [150]:
# to get join to work, let's rename country in the migration dataset

df_migration_subset["country"].replace(to_replace={
    "United States of America": "United States", 
    "Syrian Arab Republic": "Syria", 
    "Russian Federation": "Russia", 
    "Republic of Korea": "South Korea",
    "Venezuela (Bolivarian Republic of)": "Venezuela",
    "Viet Nam": "Vietnam",
    "China, Taiwan Province of China": "Taiwan",
    
}, inplace=True)

/Users/krr024/opt/anaconda3/lib/python3.9/site-packages/pandas/core/generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [151]:
# try the join again
df_weather_migration = df_weather.merge(df_migration_subset, how="left", left_on="country", right_on="country")

In [152]:
df_weather_migration.head()

,ip,status,country,countryCode,region,regionName,city,zip,lat,lon,...,isp,org,as,query,temperature,weathercode,windspeed,winddirection,time,migration_rate
0,162.27.156.161,success,United Kingdom,GB,ENG,England,London,W1B,51.5074,-0.127758,...,Daisy Corporate Services Trading Ltd,R.R. Donnelley & Sons Company,AS5413 Daisy Corporate Services Trading Ltd,162.27.156.161,12.7,0,11.2,269,2022-03-17T17:00,3.898
1,161.204.84.119,success,United States,US,CA,California,San Francisco,94128,37.7749,-122.419000,...,Visa International Service Association,Visa International Service Association,,161.204.84.119,10.8,3,9.7,243,2022-03-17T17:00,2.929
2,15.204.187.139,success,United States,US,VA,Virginia,Reston,20190,38.9609,-77.342900,...,OVH SAS,OVH US LLC,AS16276 OVH SAS,15.204.187.139,12.9,80,10.8,359,2022-03-17T17:00,2.929
3,21.5.32.123,success,United States,US,OH,Ohio,Whitehall,43218,39.9747,-82.894700,...,DoD Network Information Center,DoD Network Information Center,AS749 DoD Network Information Center,21.5.32.123,18.8,0,5.4,252,2022-03-17T17:00,2.929
4,193.71.21.8,success,Norway,NO,30,Viken,Auli,1929,60.0421,11.365500,...,GLOBALCONNECT,BROADNET,AS2116 GLOBALCONNECT AS,193.71.21.8,3.8,61,21.7,150,2022-03-17T17:00,5.273


In [153]:
# now all of the records should have a match

df_weather_migration.loc[df_weather_migration["migration_rate"].isna(), "country"].unique()

array([], dtype=object)